In [ ]:
#Projeto de Transfer Learning com Gatos e Cachorros

Notebook para classificar imagens de gatos e cachorros usando um modelo pré-treinado.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

#Tamanho e lote
IMG_SIZE = 160
BATCH_SIZE = 32

In [ ]:
#Dataset: treino e validação (80% / 20%)
(train_data_raw, val_data_raw), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
#Formatação
def format_image(image, label):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1  #Pixels [-1, 1]
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

#Aplicação nos dados
train_data = train_data_raw.map(format_image)
val_data = val_data_raw.map(format_image)

#Lotes
train_batches = train_data.shuffle(1000).batch(BATCH_SIZE)
val_batches = val_data.batch(BATCH_SIZE)

print("Dados preparados.")

In [ ]:
#Modelo-base MobileNetV2, sem camada de classificação final
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')

#Impede que o modelo base treine seus pesos
base_model.trainable = False

#Modelo final
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#Compilação
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

In [ ]:
#Épocas
epochs = 5

#Treinamento
history = model.fit(train_batches,
                    epochs=epochs,
                    validation_data=val_batches)

In [ ]:
#Acurácia e validação do histórico
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

#Perdas de treino e validação
loss = history.history['loss']
val_loss = history.history['val_loss']

#Gráfico Acurácia
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Acurácia Treino')
plt.plot(val_acc, label='Acurácia Validação')
plt.legend()
plt.title('Acurácia por Época')

#Gráfico Perda
plt.subplot(1, 2, 2)
plt.plot(loss, label='Perda Treino')
plt.plot(val_loss, label='Perda Validação')
plt.legend()
plt.title('Perda por Época')

plt.show()

In [ ]:
#Teste com Lote de Imagens
for image_batch, label_batch in val_batches.take(1):
    pass

#Previsão
predictions = model.predict(image_batch)

#Nomes das classes
class_names = ['Gato', 'Cachorro']

#Quatro imagens x previsões
plt.figure(figsize=(10, 10))
for i in range(4):
    ax = plt.subplot(2, 2, i + 1)

    #Imagem para visualização
    img_to_show = (image_batch[i].numpy() + 1) / 2.0
    plt.imshow(img_to_show)

    true_label = class_names[label_batch[i]]

    #Previsão < 0.5 = Gato, Previsão >= 0.5 = Cachorro
    prediction_score = predictions[i][0]
    predicted_label = class_names[int(prediction_score > 0.5)]

    plt.title(f"Real: {true_label}\nPrevisto: {predicted_label}")
    plt.axis("off")